In [1]:
import pandas as pd
import numpy as np
from data_analysis import get_heart_bounds, correlation_coef, graph_3d, get_index, lv_rv
from graph import *
from BayesOptLib.bayes_opt.bayesian_optimization import BayesianOptimization
from RandomSampler import RandomSampler
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics.pairwise import euclidean_distances
matplotlib.use('Qt5Agg')  # or can use 'TkAgg', whatever you have/prefer
# from prettytable import PrettyTable
import plotly.graph_objects as go
%matplotlib inline 

ecgs = pd.read_csv("new_simu-data/Heart1/Heart1_SimuData_4000.csv", header=None).to_numpy()
labels = pd.read_csv("new_simu-data/Heart1/Coord1_4000.csv", header=None).to_numpy() / 1000
uvc = pd.read_csv("new_simu-data/UVC/Coord1_UVC_4000.csv", header=None).to_numpy()
idx=lv_rv(uvc,-1)
uvc=uvc[idx]
ecgs = ecgs[idx]
labels = labels[idx]

In [2]:
def black_box(x, y, z):
    """
    Represents a black box function to maximize for CC of two ECGs given an XYZ coordinate
    :param x:
    :param y: coordinates of the prediction
    :param z:
    :return: CC of that point and the target
    """
    sample_ecg = ecgs[get_index(np.array([x, y, z]),labels)]
    return abs(correlation_coef(target_ecg, sample_ecg))

In [52]:
for j in range(1):
    print(j)
    # bounds = get_heart_bounds(labels) # getting bounds 
    tidx = np.random.randint(0, labels.shape[0])    # Pick out a sample to use as a target
    _, target_ecg,target_uvc = labels[tidx], ecgs[tidx], uvc[tidx]
    labels = uvc[:,0:3]
    plotly_plot(target_ecg,labels,ecgs)

0


In [14]:
def gp_uvc(gp,target,labels):
    color_gradient = []
    for i in range(len(labels)):
        cc = gp.predict(labels[i].reshape(1,-1),return_std=False)
        color_gradient.append(cc)
    color_gradient = np.array(color_gradient).flatten()
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(x=labels[:,0],y=labels[:,1],z=labels[:,2],mode='markers',marker=dict(size=5,color=color_gradient,colorbar=dict(
                title=""),colorscale='rainbow',opacity=0)))
    fig.add_trace(go.Scatter3d(x=[target[0]],y=[target[1]],z=[target[2]],mode = 'markers',marker = dict(size=10,color='Black',symbol= 'x')))
    fig.show()

In [11]:
init = 5 #initial random sample
steps = 15 # total AL step
af = "ucb" 
bounds = get_heart_bounds(labels)

In [53]:
optimizer = BayesianOptimization(f=black_box,pbounds=bounds,random_state=None, real_set=labels)
gp,X = optimizer.gpfit(init_points=init, n_iter=steps,  acq=af, kappa = 1,kappa_decay=0.75,kappa_decay_delay=2)

|   iter    |  target   |     x     |     y     |     z     |
-------------------------------------------------------------
|  1        |  0.5829   |  0.7778   |  0.5556   |  1.818    |
|  2        |  0.3578   |  1.0      |  0.6667   |  0.4958   |
|  3        |  0.2103   |  0.2222   |  0.8889   | -0.1653   |
|  4        |  0.05339  |  0.0      |  0.5556   |  0.8263   |
|  5        |  0.3578   |  0.4444   |  0.1111   |  0.4958   |
|  6        |  0.6451   |  0.8889   |  0.5556   |  1.818    |
|  7        |  0.8053   |  1.0      |  0.5556   |  2.148    |
|  8        |  0.8517   |  1.0      |  0.4444   |  2.479    |
|  9        |  0.9667   |  1.0      |  0.5556   |  3.14     |
|  10       |  0.9736   |  1.0      |  1.0      |  3.14     |
|  11       |  0.8934   |  1.0      |  0.0      |  3.14     |
|  12       |  0.9792   |  1.0      |  0.7778   |  3.14     |
|  13       |  0.9768   |  1.0      |  0.8889   |  3.14     |
|  14       |  0.9454   |  1.0      |  0.8889   |  2.809    |
|  15   

In [54]:
gp_uvc(gp,target_uvc,labels)

In [49]:
plotly_plot(target_ecg,labels,ecgs)